# Project 3 
## EDA

**Project Partner**: Star Li, Setfan Li

## Setup

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style = "whitegrid", 
        color_codes = True,
        font_scale = 1.5)

In [2]:
data_state = pd.read_csv('4.18states.csv')

FileNotFoundError: [Errno 2] File b'4.18states.csv' does not exist: b'4.18states.csv'

In [44]:
data_counties = pd.read_csv('abridged_couties.csv')

In [45]:
data_time_conf = pd.read_csv('time_series_covid19_confirmed_US.csv')

In [46]:
data_time_de = pd.read_csv('time_series_covid19_deaths_US.csv')

## Data Filtering (we only want U.S. data)

In [47]:
data_state.head(5)

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
0,Alabama,US,2020-04-18 22:32:47,32.3182,-86.9023,4712,153,NaN,4559.0,1.0,100.492717,42538.0,620.0,3.247029,84000001,USA,907.206961,13.157895
1,Alaska,US,2020-04-18 22:32:47,61.3707,-152.4044,314,9,147.0,305.0,2.0,52.530410,9655.0,39.0,2.866242,84000002,USA,1615.226458,12.420382
2,American Samoa,US,NaN,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-18 22:32:47,33.7298,-111.4312,4724,180,539.0,4544.0,4.0,64.901548,51045.0,566.0,3.810330,84000004,USA,701.291175,11.981372
4,Arkansas,US,2020-04-18 22:32:47,34.9697,-92.3731,1744,38,703.0,1706.0,5.0,67.361213,24141.0,291.0,2.178899,84000005,USA,932.435235,16.685780


In [48]:
data_state = data_state.loc[data_state['Country_Region'] == 'US']

In [49]:
data_state = data_state.rename(columns={"Long_": "Long"})

## Data Cleaning

### 1. "4.18state"

- Divide the data into edge case (placeselike Diamond Princess that does not have lat and long) and general case

In [50]:
edge_case = data_state[data_state['Lat'].isnull()]

In [51]:
edge_case.head(10)

,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
9,Diamond Princess,US,2020-04-18 22:32:47,NaN,NaN,49,0,0.0,49.0,88888.0,NaN,NaN,NaN,0.0,84088888,USA,NaN,NaN
13,Grand Princess,US,2020-04-18 22:32:47,NaN,NaN,103,0,0.0,103.0,99999.0,NaN,NaN,NaN,0.0,84099999,USA,NaN,NaN
116,Recovered,US,NaN,NaN,NaN,0,0,64840.0,-64840.0,NaN,NaN,NaN,NaN,NaN,84070001,USA,NaN,NaN


In [52]:
general_state = data_state[data_state['Lat'].notnull()]

- We know that, based on the provided README file, Active cases = total confirmed - total recovered - total deaths, we can safely fill out all the NaN's in Deaths, Recorvered, and Active with 0, since they are all mutually exclusive.

In [53]:
general_state = data_state[data_state['Lat'].notnull()]

- We know that, based on the provided README file, Active cases = total confirmed - total recovered - total deaths, we can safely fill out all the NaN's in Deaths, Recorvered, and Active with 0, since they are all mutually exclusive.

In [54]:
general_state[['Recovered']] = general_state[['Recovered']].fillna(0)

/srv/conda/envs/data100/lib/python3.7/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [55]:
general_state[['Active']] = general_state[['Active']].fillna(0)

In [56]:
general_state[['Mortality_Rate']] = general_state[['Mortality_Rate']].fillna(0)

- Notice that US Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases, so if the number of People_Hospitalized is NaN or 0, we can logically fill in 0 for all the NaN's in these two columns

In [57]:
general_state[['People_Hospitalized']] = general_state[['People_Hospitalized']].fillna(0)

In [58]:
general_state[['Hospitalization_Rate']] = general_state[['Hospitalization_Rate']].fillna(0)

- As we can that we only have one na value left

In [59]:
general_state.isna().sum()

Province_State          0
Country_Region          0
Last_Update             1
Lat                     0
Long                    0
Confirmed               0
Deaths                  0
Recovered               0
Active                  0
FIPS                    0
Incident_Rate           0
People_Tested           0
People_Hospitalized     0
Mortality_Rate          0
UID                     0
ISO3                    0
Testing_Rate            0
Hospitalization_Rate    0
dtype: int64

In [60]:
general_state[general_state['Last_Update'].isnull()]

,Province_State,Country_Region,Last_Update,Lat,Long,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3,Testing_Rate,Hospitalization_Rate
2,American Samoa,US,NaN,-14.271,-170.132,0,0,0.0,0.0,60.0,0.0,3.0,0.0,0.0,16,ASM,5.391708,0.0


- Since "American Samoa" has such a small number of cases, it will have relatively small effect on our prediction, so we decided to remove this area to keep the consistency of our dataframe.

In [61]:
general_state = general_state[general_state['Last_Update'].notnull()]

### 2. Join "abridged_couties" with "time_series_covid19_deaths_US"

- abridged_counties contains wonderful information about population health condition as well as population demograpics, and time_series_covid19_deaths_US has a wonderful pattern of the times series of confirms in the U.S., so it would be a lot easier to select feature from this join table rather than subseting data separately from each data set when selecting features.

In [62]:
data_time_de['UID'] = data_time_de['UID'].astype(int)

In [63]:
data_counties = data_counties.iloc[:-2, :]

In [64]:
data_counties["countyFIPS"] = data_counties['countyFIPS'].astype(int)

In [65]:
data_counties['countyFIPS'] = data_counties['countyFIPS'] + 84000000

In [66]:
combined_dead = data_time_de.merge(data_counties, left_on= 'UID', right_on = 'countyFIPS')

### 2. Join "abridged_couties" with "time_series_covid19_confirmed_US"

- abridged_counties contains wonderful information about population health condition as well as population demograpics, and time_series_covid19_confirmed_US has a wonderful pattern of the times series of confirms in the U.S., so it would be a lot easier to select feature from this join table rather than subseting data separately from each data set when selecting features.

In [67]:
data_time_conf['UID'] = data_time_conf['UID'].astype(int)

In [68]:
combined_conf = data_time_conf.merge(data_counties, left_on= 'UID', right_on = 'countyFIPS')

### 3. Clean out unwanted columns for EDA

In [69]:
all_in_1 = combined_conf.merge(general_state, left_on = 'Province_State', right_on='Province_State')

In [70]:
drop_columns = ['UID_x', 'iso2', 'iso3', "code3", "FIPS_x", 'Admin2', 
                'Province_State', "Country_Region_x", "Combined_Key",
               "StateName", "State", 'countyFIPS', 'STATEFP', 'COUNTYFP', 'CountyName', 'lat', 
                'lon', 'POP_LATITUDE', 'POP_LONGITUDE', 'FIPS_y', "UID_y", 
                "ISO3", 'CensusRegionName', 'Country_Region_y', 'Last_Update', 'CensusDivisionName']

In [71]:
all_in_1 = all_in_1.drop(columns = drop_columns)

### 4. Train test split

In [72]:
y = all_in_1['4/18/20']

In [73]:
X = all_in_1.drop(columns = ['4/18/20'])

In [74]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

### 5. Use SKlearn Feature Selection package to create a df filter

In [75]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent")
X_train = imp.fit_transform(X_train)

In [80]:
filled_df = pd.DataFrame(X_train,
                   columns=all_in_1.drop(columns = ['4/18/20']).columns)

In [82]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(filled_df, y_train)
feature_filter = SelectFromModel(clf, prefit=True)
X_train = feature_filter.transform(X_train)

### 6. Decision Tree Regression Model fitting

In [83]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=4)
regr_1.fit(X_train, y_train);

In [84]:
training_accuracy = regr_1.score(X_train, y_train)
print("Training Accuracy: ", training_accuracy)

Training Accuracy:  0.997405112981654


### 7. Test Accuracy

In [39]:
X_test = imp.fit_transform(X_test)

In [40]:
X_test = feature_filter.transform(X_test)

In [41]:
training_accuracy = regr_1.score(X_test, y_test)
print("Test Accuracy: ", training_accuracy)

Test Accuracy:  0.9766697497487835


### 7. The selected features by SKLearn

In [81]:
filled_df

,Lat_x,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,Testing_Rate,Hospitalization_Rate
0,33.176597,-98.687909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18704.0,476.0,4806.0,18228.0,81.376364,176239.0,1321.0,2.544910,766.771225,7.062660
1,35.965756,-77.984225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6328.0,187.0,0.0,6141.0,63.803990,76211.0,388.0,2.955120,768.420655,6.131479
2,30.265487,-85.621226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25492.0,748.0,0.0,24744.0,120.060628,246527.0,3842.0,2.934254,1161.077449,15.071395
3,38.594719,-83.821510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2707.0,144.0,979.0,2563.0,79.075676,30596.0,1008.0,5.319542,893.756702,37.236793
4,39.391335,-88.806111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,29160.0,1259.0,0.0,27901.0,248.473604,137404.0,4340.0,4.317558,1170.825347,14.883402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,44.718503,-92.422442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4199.0,212.0,0.0,3987.0,81.146824,48161.0,1176.0,5.048821,930.724499,28.006668
2822,36.993084,-85.060701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2707.0,144.0,979.0,2563.0,79.075676,30596.0,1008.0,5.319542,893.756702,37.236793
2823,30.844254,-91.049767,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23580.0,1267.0,0.0,22313.0,512.913545,137999.0,1761.0,5.373198,3001.762352,7.468193
2824,44.334810,-84.127252,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30791.0,2308.0,3237.0,28483.0,386.469173,99727.0,3634.0,7.495697,1251.710281,11.802150


In [90]:
subset = feature_filter.get_support()

In [91]:
filled_df = filled_df.loc[:, subset]

In [93]:
filled_df.columns

Index(['Lat_x', 'Long_', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20',
       '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20',
       '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20',
       '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20',
       'Rural-UrbanContinuumCode2013', 'PopulationEstimate2018',
       'PopTotalMale2017', 'PopTotalFemale2017', 'FracMale2017',
       'PopulationEstimate65+2017', 'PopulationDensityperSqMile2010',
       'CensusPopulation2010', 'MedianAge2010', '#EligibleforMedicare2018',
       'MedicareEnrollment,AgedTot2017', 'DiabetesPercentage',
       'HeartDiseaseMortality', 'StrokeMortality', 'Smokers_Percentage',
       'RespMortalityRate2014', '#FTEHospitalTotal2017',
       'TotalM.D.'s,TotNon-FedandFed2017', '#Hospitals', '#ICU_beds',
       'dem_to_rep_ratio', 'PopMale<52010', 'PopFmle<52010', 'PopMale5-92010',
       'PopFmle5-92010', 'PopMale10-142010', 'PopFmle10-142010',
       'PopMale1